# Generating Station Buffers

We are creating a 20-minute walk buffer for each station, We assume that this is the region from which people are likely to use the transit stop. Hence changes to income in this buffer is of importance for the study.

In [9]:
# to read and wrangle data
import pandas as pd

# for spatial data
import geopandas as gpd

# for plotting
import matplotlib.pyplot as plt

# for network analysis0-yujee
import networkx as nx

# for street network analysis
import osmnx as ox

# for basemaps
import contextily as ctx


from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import Polygon

In [10]:
stations=gpd.read_file("stations.geojson")

In [11]:
stations

,Station,City,Code,TOD,County,Marker,geometry
0,ASHBY,Berkeley,BKY,Yes,ALA,D,POINT (-122.27012 37.85321)
1,MACARTHUR,Oakland,OAK,Yes,ALA,D,POINT (-122.26712 37.82871)
2,HAYWARD,Hayward,HAY,Yes,ALA,D,POINT (-122.08720 37.67012)
3,SOUTH HAYWARD,Hayward,HAY,Yes,ALA,D,POINT (-122.05704 37.63448)
4,FRUITVALE,Oakland,OAK,Yes,ALA,D,POINT (-122.22420 37.77456)
5,NORTH BERKELEY,Berkeley,BKY,No,ALA,o,POINT (-122.28335 37.87406)
6,19TH STREET/OAKLAND,Oakland,OAK,No,ALA,o,POINT (-122.26839 37.80808)
7,BAY FAIR,San Leandro,SAN,No,ALA,o,POINT (-122.12706 37.69757)
8,UNION CITY,Union City,UNC,No,ALA,o,POINT (-122.01715 37.59087)
9,LAKE MERRITT,Oakland,OAK,No,ALA,o,POINT (-122.26554 37.79768)


We generate isochrones for the 20 minute walk distance and this will become our buffer area. 

In [12]:
network_type = "walk"
trip_time = 20  # in minutes

polygon_bound=[]
for i in range(len(stations.geometry)):
    point=(stations.geometry[i].y,stations.geometry[i].x)

    G = ox.graph_from_point(point, network_type=network_type) 
    gdf_nodes = ox.graph_to_gdfs(G, edges=False)
    x, y = gdf_nodes["geometry"].unary_union.centroid.xy
    G = ox.project_graph(G, to_crs='epsg:4326')

    center_node = ox.distance.nearest_nodes(G, x[0], y[0])

    subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance="time")
    node_points = [Point((data["x"], data["y"])) for node, data in subgraph.nodes(data=True)]
    bounding_poly = gpd.GeoSeries(node_points).unary_union.convex_hull
    polygon_bound.append(bounding_poly)
    

stations['buffer_polygon']=polygon_bound
stations

,Station,City,Code,TOD,County,Marker,geometry,buffer_polygon
0,ASHBY,Berkeley,BKY,Yes,ALA,D,POINT (-122.27012 37.85321),"POLYGON ((-122.2705249 37.8442736, -122.277355..."
1,MACARTHUR,Oakland,OAK,Yes,ALA,D,POINT (-122.26712 37.82871),"POLYGON ((-122.2630119 37.820378, -122.2707559..."
2,HAYWARD,Hayward,HAY,Yes,ALA,D,POINT (-122.08720 37.67012),"POLYGON ((-122.0853634 37.6611423, -122.089055..."
3,SOUTH HAYWARD,Hayward,HAY,Yes,ALA,D,POINT (-122.05704 37.63448),"POLYGON ((-122.0642612 37.6259094, -122.066706..."
4,FRUITVALE,Oakland,OAK,Yes,ALA,D,POINT (-122.22420 37.77456),"POLYGON ((-122.2167742 37.7656538, -122.221195..."
5,NORTH BERKELEY,Berkeley,BKY,No,ALA,o,POINT (-122.28335 37.87406),"POLYGON ((-122.2865786 37.8651654, -122.291102..."
6,19TH STREET/OAKLAND,Oakland,OAK,No,ALA,o,POINT (-122.26839 37.80808),"POLYGON ((-122.2732646 37.7991381, -122.275865..."
7,BAY FAIR,San Leandro,SAN,No,ALA,o,POINT (-122.12706 37.69757),"POLYGON ((-122.1295769 37.6886827, -122.130019..."
8,UNION CITY,Union City,UNC,No,ALA,o,POINT (-122.01715 37.59087),"POLYGON ((-122.022147 37.5818938, -122.024772 ..."
9,LAKE MERRITT,Oakland,OAK,No,ALA,o,POINT (-122.26554 37.79768),"POLYGON ((-122.2678111 37.7904618, -122.275731..."


These plots are still of the stations, and not the buffers. To examine why let us check the datatypes.

In [13]:
stations.to_csv('final_stations_walk.csv')